In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Stochastic Market Lab — GBM, GARCH & Monte Carlo
================================================

What this script does
---------------------
1) Pulls daily data for tickers (or use your own DataFrame).
2) Estimates GBM parameters (drift 'mu' and vol 'sigma') from log returns.
3) Fits GARCH(1,1) to returns to forecast next-day/week volatility (fallback to EWMA).
4) Simulates price paths with Monte Carlo:
   - Constant-vol GBM
   - Time-varying vol (using GARCH forecast for the next H days)
5) Produces probability-based outputs:
   - Next-day & next-week price range percentiles
   - Probability of hitting target/stop
   - Simple probability-based trade decision (long/short/flat)
6) Generates synthetic paths to test strategies offline.

Dependencies
------------
- numpy, pandas, yfinance, matplotlib (optional plotting)
- arch (optional: for GARCH; falls back to EWMA if not available)

Tip
---
If you can't install 'arch', the script will use EWMA volatility ('lambda'=0.94) for forecasting.
"""

from __future__ import annotations
import os, math, warnings
from dataclasses import dataclass, asdict
from typing import Dict, Optional, Tuple

import numpy as np
import pandas as pd

# Optional GARCH
try:
    from arch import arch_model
    ARCH_AVAILABLE = True
except Exception:
    ARCH_AVAILABLE = False
    warnings.warn("arch package not found; falling back to EWMA volatility forecasting.")

# Optional Yahoo Finance
try:
    import yfinance as yf
    YF_AVAILABLE = True
except Exception:
    YF_AVAILABLE = False
    warnings.warn("yfinance not found; pass your own DataFrame to run without downloads.")

# -------------------------
# Configuration
# -------------------------
@dataclass
class Config:
    tickers: tuple = ("FEDERALBNK.NS", "HINDALCO.NS")     # NSE Yahoo tickers; replace as needed
    start: str = "2018-01-01"
    end:   str = None                              # None = today
    price_col: str = "Adj Close"                   # Which column to use
    seed: int = 42

    # Forecast horizons (trading days)
    horizon_days_next: int = 1                     # next day
    horizon_days_week: int = 5                     # ~1 week

    # Monte Carlo settings
    n_paths: int = 20_000
    use_garch_vol: bool = True                     # If True and arch available -> use GARCH-based vol forecast
    ewma_lambda: float = 0.94                      # For EWMA fallback

    # Probability-based trade decision
    target_up_pct: float = 0.01                    # +1% target
    stop_down_pct: float = 0.008                   # -0.8% stop
    go_long_threshold: float = 0.55                # need >= 55% prob of reaching target before stop
    go_short_threshold: float = 0.45               # <= 45% prob => short; else flat

    # Output
    out_dir: str = "outputs_stochastic_lab"
    make_plots: bool = False                       # Set True to see histograms/paths
    save_csv: bool = True

CFG = Config()


# -------------------------
# Utilities
# -------------------------
def ensure_dir(path: str) -> None:
    if path and not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

def log_returns(prices: pd.Series) -> pd.Series:
    return np.log(prices).diff().dropna()

def annualize(mu_daily: float, sigma_daily: float, trading_days: int = 252) -> Tuple[float, float]:
    return mu_daily * trading_days, sigma_daily * math.sqrt(trading_days)


# -------------------------
# Data
# -------------------------
def get_prices_yf(ticker: str, start: str, end: Optional[str], price_col: str) -> pd.Series:
    if not YF_AVAILABLE:
        raise RuntimeError("yfinance not available. Install it or pass your own price series.")
    df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
    if df.empty:
        raise ValueError(f"No data for {ticker}.")
    if price_col not in df.columns:
        fallback = "Adj Close" if "Adj Close" in df.columns else "Close"
        warnings.warn(f"{price_col} not found in Yahoo frame; using {fallback}.")
        price_col = fallback
    return df[price_col].dropna()


# -------------------------
# GBM Estimation
# -------------------------
def estimate_gbm_params(returns: pd.Series) -> Tuple[float, float]:
    """
    Estimate GBM parameters from log returns:
    r_t ~ N(mu*dt, sigma^2*dt). Using dt=1 day:
    mu_hat = mean(r), sigma_hat = std(r).
    """
    mu_hat = float(returns.mean())
    sigma_hat = float(returns.std(ddof=1))
    return mu_hat, sigma_hat


# -------------------------
# GARCH Forecast (or EWMA)
# -------------------------
def forecast_vol_garch(returns: pd.Series, horizon_days: int, ewma_lambda: float) -> np.ndarray:
    """
    Returns array of length 'horizon_days' with daily sigmas (std dev).
    If arch not available, returns EWMA-based constant forecast.
    """
    if ARCH_AVAILABLE:
        r = (returns - returns.mean()).dropna()
        am = arch_model(r * 100, vol="GARCH", p=1, q=1, dist="normal", mean="zero")  # scale to %
        res = am.fit(disp="off")
        fcast = res.forecast(horizon=horizon_days, reindex=False)
        var = fcast.variance.values[-1] / (100**2)  # back to decimal
        sigma = np.sqrt(var)
        sigma = np.array(sigma, dtype=float).reshape(-1)
        if sigma.size == 1:
            sigma = np.repeat(sigma, horizon_days)
        return sigma
    else:
        # EWMA fallback
        r = returns.values
        if r.size < 2:
            raise ValueError("Not enough returns for EWMA.")
        var = np.var(r, ddof=1)
        for x in r[::-1][:50]:  # fold in recent 50 obs to stabilize
            var = ewma_lambda * var + (1 - ewma_lambda) * x * x
        sigma = math.sqrt(var)
        return np.repeat(sigma, horizon_days)


# -------------------------
# Monte Carlo Simulators
# -------------------------
def simulate_gbm_paths(S0: float, mu: float, sigma: float, steps: int, n_paths: int, seed: int) -> np.ndarray:
    """
    Simulate GBM with constant mu, sigma:
    S_{t+1} = S_t * exp((mu - 0.5*sigma^2)*dt + sigma*sqrt(dt)*Z)
    dt = 1 (daily)
    Returns array shape (steps+1, n_paths)
    """
    rng = np.random.default_rng(seed)
    dt = 1.0
    drift = (mu - 0.5 * sigma * sigma) * dt
    shock_scale = sigma * math.sqrt(dt)
    Z = rng.standard_normal(size=(steps, n_paths))
    increments = drift + shock_scale * Z
    log_paths = np.vstack([np.zeros(n_paths), np.cumsum(increments, axis=0)])
    return S0 * np.exp(log_paths)


def simulate_paths_with_vol_curve(S0: float, mu: float, sigmas: np.ndarray, n_paths: int, seed: int) -> np.ndarray:
    """
    Like GBM but allows a per-step sigma (e.g., GARCH forecast curve).
    """
    rng = np.random.default_rng(seed)
    steps = len(sigmas)
    Z = rng.standard_normal(size=(steps, n_paths))
    increments = (mu - 0.5 * sigmas**2)[:, None] + (sigmas[:, None] * Z)
    log_paths = np.vstack([np.zeros(n_paths), np.cumsum(increments, axis=0)])
    return S0 * np.exp(log_paths)


# -------------------------
# Probability-based decisions
# -------------------------
def probability_of_hitting_target_before_stop(
    S0: float,
    mu: float,
    sigma: float,
    target_up_pct: float,
    stop_down_pct: float
) -> float:
    """
    Approximate probability of hitting an upper barrier (S0*(1+u)) before a lower
    barrier (S0*(1-d)) under drifted Brownian motion in log space.
    """
    u = math.log(1 + target_up_pct)
    d = math.log(1 - stop_down_pct)
    if d >= 0:
        raise ValueError("stop_down_pct must be >0 and <1.")
    a = -d
    b = u
    if abs(mu) < 1e-9:
        return a / (a + b)
    else:
        num = 1 - math.exp(-2 * mu * a / (sigma * sigma))
        den = 1 - math.exp(-2 * mu * (a + b) / (sigma * sigma))
        return max(0.0, min(1.0, num / den))


def percentile_ranges_from_paths(paths: np.ndarray, percentiles=(5, 50, 95)) -> Dict[int, float]:
    terminal = paths[-1]
    return {p: float(np.percentile(terminal, p)) for p in percentiles}


# -------------------------
# Main per-ticker workflow
# -------------------------
def analyze_ticker(ticker: str, cfg: Config) -> Dict:
    series = get_prices_yf(ticker, cfg.start, cfg.end, cfg.price_col)
    px = series.dropna()
    if px.size < 60:
        raise ValueError(f"Not enough data for {ticker}.")

    ret = log_returns(px)
    mu_d, sig_d = estimate_gbm_params(ret)
    mu_ann, sig_ann = annualize(mu_d, sig_d)

    # Vol forecast for horizons
    sig_next_curve = forecast_vol_garch(ret, cfg.horizon_days_next, cfg.ewma_lambda)
    sig_week_curve = forecast_vol_garch(ret, cfg.horizon_days_week, cfg.ewma_lambda)

    S0 = float(px.iloc[-1])

    # Monte Carlo — constant vol (GBM)
    paths_next_gbm = simulate_gbm_paths(S0, mu_d, sig_d, cfg.horizon_days_next, cfg.n_paths, cfg.seed)
    paths_week_gbm = simulate_gbm_paths(S0, mu_d, sig_d, cfg.horizon_days_week, cfg.n_paths, cfg.seed + 1)

    # Monte Carlo — time-varying vol (GARCH or EWMA forecast)
    if cfg.use_garch_vol:
        paths_next_tv = simulate_paths_with_vol_curve(S0, mu_d, sig_next_curve, cfg.n_paths, cfg.seed + 2)
        paths_week_tv = simulate_paths_with_vol_curve(S0, mu_d, sig_week_curve, cfg.n_paths, cfg.seed + 3)
    else:
        paths_next_tv = paths_next_gbm
        paths_week_tv = paths_week_gbm

    # Ranges
    rng_next_gbm = percentile_ranges_from_paths(paths_next_gbm)
    rng_week_gbm = percentile_ranges_from_paths(paths_week_gbm)
    rng_next_tv  = percentile_ranges_from_paths(paths_next_tv)
    rng_week_tv  = percentile_ranges_from_paths(paths_week_tv)

    # Probability of target-before-stop (1-day, constant vol approx)
    p_up_first = probability_of_hitting_target_before_stop(
        S0=S0, mu=mu_d, sigma=sig_d,
        target_up_pct=cfg.target_up_pct,
        stop_down_pct=cfg.stop_down_pct
    )

    # Simple decision rule
    if p_up_first >= cfg.go_long_threshold:
        decision = "LONG"
    elif p_up_first <= cfg.go_short_threshold:
        decision = "SHORT"
    else:
        decision = "FLAT"

    result = {
        "ticker": ticker,
        "last_price": S0,
        "mu_daily": mu_d,
        "sigma_daily": sig_d,
        "mu_annual": mu_ann,
        "sigma_annual": sig_ann,
        "p_target_before_stop_1d": p_up_first,
        "decision_1d": decision,
        "range_next_gbm": rng_next_gbm,
        "range_next_tv": rng_next_tv,
        "range_week_gbm": rng_week_gbm,
        "range_week_tv": rng_week_tv,
        "garch_used": (cfg.use_garch_vol and ARCH_AVAILABLE),
        "horizon_days_next": cfg.horizon_days_next,
        "horizon_days_week": cfg.horizon_days_week,
        "n_paths": cfg.n_paths
    }
    return result


# -------------------------
# Batch runner
# -------------------------
def run(cfg: Config) -> pd.DataFrame:
    ensure_dir(cfg.out_dir)
    rows = []
    for t in cfg.tickers:
        try:
            out = analyze_ticker(t, cfg)
            rows.append({
                "ticker": out["ticker"],
                "last_price": out["last_price"],
                "mu_daily": out["mu_daily"],
                "sigma_daily": out["sigma_daily"],
                "mu_annual": out["mu_annual"],
                "sigma_annual": out["sigma_annual"],
                "p_target_before_stop_1d": out["p_target_before_stop_1d"],
                "decision_1d": out["decision_1d"],
                "range_next_gbm_p5": out["range_next_gbm"][5],
                "range_next_gbm_p50": out["range_next_gbm"][50],
                "range_next_gbm_p95": out["range_next_gbm"][95],
                "range_next_tv_p5": out["range_next_tv"][5],
                "range_next_tv_p50": out["range_next_tv"][50],
                "range_next_tv_p95": out["range_next_tv"][95],
                "range_week_gbm_p5": out["range_week_gbm"][5],
                "range_week_gbm_p50": out["range_week_gbm"][50],
                "range_week_gbm_p95": out["range_week_gbm"][95],
                "range_week_tv_p5": out["range_week_tv"][5],
                "range_week_tv_p50": out["range_week_tv"][50],
                "range_week_tv_p95": out["range_week_tv"][95],
                "garch_used": out["garch_used"]
            })
        except Exception as e:
            rows.append({
                "ticker": t,
                "error": str(e)
            })
    df = pd.DataFrame(rows)
    if cfg.save_csv:
        path = os.path.join(cfg.out_dir, "stochastic_market_summary.csv")
        df.to_csv(path, index=False)
    return df


# -------------------------
# Optional plotting helpers
# -------------------------
def demo_plot(paths: np.ndarray, title: str = "Sample Paths"):
    import matplotlib.pyplot as plt
    idx = np.linspace(0, paths.shape[0]-1, num=paths.shape[0], dtype=int)
    plt.figure(figsize=(8,4.5))
    for i in range(min(50, paths.shape[1])):
        plt.plot(idx, paths[:, i], alpha=0.3)
    plt.title(title)
    plt.xlabel("Step (days)")
    plt.ylabel("Price")
    plt.tight_layout()
    plt.show()


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    np.set_printoptions(precision=4, suppress=True)
    pd.options.display.float_format = "{:,.6f}".format

    print("=== Config ===")
    print(asdict(CFG))

    df_summary = run(CFG)

    print("\n=== Stochastic Summary ===")
    # Cosmetic: hide "error" column if present when printing
    to_show = df_summary.drop(columns=["error"], errors="ignore")
    with pd.option_context("display.max_columns", None):
        print(to_show)

    # ---------- FIXED SECTION (no mixed row/column masks) ----------
    # Example: show interpretation guidance for the first non-error row
    has_error_col = "error" in df_summary.columns
    if has_error_col:
        valid_mask = df_summary["error"].isna()
    else:
        valid_mask = df_summary["ticker"].notna()

    valid = df_summary.loc[valid_mask]

    if valid.empty:
        print("\n(No valid rows — check the 'error' column above.)")
    else:
        first = valid.iloc[0]
        print("\nHow to use:")
        print("- 'range_next_*_p5/p50/p95' give the 5/50/95th percentiles of the forecast distribution.")
        print(f"- 'p_target_before_stop_1d' is the probability that a +{int(CFG.target_up_pct*100)}% target "
              f"is hit before a -{int(CFG.stop_down_pct*100)}% stop within 1 day.")
        print("- 'decision_1d' maps that probability into LONG/SHORT/FLAT based on your thresholds.")
        print("- Set 'use_garch_vol=True' (and install 'arch') to account for volatility clustering.")
        print("- Use the path simulators to create synthetic data for strategy stress-testing.\n")

        # A small, concrete example readout for the first valid row:
        fields = [
            ("Ticker", "ticker"),
            ("Last price", "last_price"),
            ("Daily μ (drift)", "mu_daily"),
            ("Daily σ", "sigma_daily"),
            ("Next-day 5/50/95 (GBM)", None),
            ("Next-day 5/50/95 (TV)", None),
            ("1-day P(target before stop)", "p_target_before_stop_1d"),
            ("Decision", "decision_1d"),
        ]
        print("Example interpretation for:", first["ticker"])
        print(f"- Last price: {first.get('last_price', float('nan')):.4f}")
        print(f"- Daily μ (drift): {first.get('mu_daily', float('nan')):.6f}, "
              f"daily σ: {first.get('sigma_daily', float('nan')):.6f}")
        try:
            print(f"- Next-day 5/50/95 (GBM): {first['range_next_gbm_p5']:.2f} / "
                  f"{first['range_next_gbm_p50']:.2f} / {first['range_next_gbm_p95']:.2f}")
        except Exception:
            pass
        try:
            print(f"- Next-day 5/50/95 (TV):  {first['range_next_tv_p5']:.2f} / "
                  f"{first['range_next_tv_p50']:.2f} / {first['range_next_tv_p95']:.2f}")
        except Exception:
            pass
        print(f"- 1-day P(target before stop): {first.get('p_target_before_stop_1d', float('nan')):.3f} "
              f"→ Decision: {first.get('decision_1d', 'NA')}")

    if CFG.make_plots:
        # As a small demo, re-simulate one ticker for visualization
        t0 = CFG.tickers[0]
        series = get_prices_yf(t0, CFG.start, CFG.end, CFG.price_col)
        ret = log_returns(series)
        mu_d, sig_d = estimate_gbm_params(ret)
        S0 = float(series.iloc[-1])
        # Constant vol GBM sample
        paths = simulate_gbm_paths(S0, mu_d, sig_d, steps=CFG.horizon_days_week, n_paths=500, seed=CFG.seed)
        demo_plot(paths, title=f"{t0} — GBM sample paths (week)")


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_52346/810966028.py:46: UserWarning: arch package not found; falling back to EWMA volatility forecasting.
  warnings.warn("arch package not found; falling back to EWMA volatility forecasting.")


=== Config ===
{'tickers': ('FEDERALBNK.NS', 'HINDALCO.NS'), 'start': '2018-01-01', 'end': None, 'price_col': 'Adj Close', 'seed': 42, 'horizon_days_next': 1, 'horizon_days_week': 5, 'n_paths': 20000, 'use_garch_vol': True, 'ewma_lambda': 0.94, 'target_up_pct': 0.01, 'stop_down_pct': 0.008, 'go_long_threshold': 0.55, 'go_short_threshold': 0.45, 'out_dir': 'outputs_stochastic_lab', 'make_plots': False, 'save_csv': True}


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_52346/810966028.py:129: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mu_hat = float(returns.mean())
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_52346/810966028.py:130: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma_hat = float(returns.std(ddof=1))
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_52346/810966028.py:161: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma = math.sqrt(var)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_52346/810966028.py:247: FutureWarning: Calling float on a single element Series is dep


=== Stochastic Summary ===
          ticker  last_price  mu_daily  sigma_daily  mu_annual  sigma_annual  \
0  FEDERALBNK.NS  237.889999  0.000441     0.022875   0.111085      0.363124   
1    HINDALCO.NS  847.200012  0.000616     0.024100   0.155232      0.382571   

   p_target_before_stop_1d decision_1d  range_next_gbm_p5  range_next_gbm_p50  \
0                 0.450412        FLAT         229.152251          237.923304   
1                 0.451383        FLAT         814.559751          847.440905   

   range_next_gbm_p95  range_next_tv_p5  range_next_tv_p50  range_next_tv_p95  \
0          247.199491        233.445327         237.993653         242.567696   
1          882.286393        830.259913         847.712490         865.290027   

   range_week_gbm_p5  range_week_gbm_p50  range_week_gbm_p95  \
0         218.630460          238.102501          258.689009   
1         775.628810          848.577635          926.049716   

   range_week_tv_p5  range_week_tv_p50  range_week

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_52346/810966028.py:129: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mu_hat = float(returns.mean())
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_52346/810966028.py:130: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma_hat = float(returns.std(ddof=1))
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_52346/810966028.py:161: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma = math.sqrt(var)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_52346/810966028.py:247: FutureWarning: Calling float on a single element Series is dep